In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [125]:
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
all_index=df1.index.to_numpy()
last_row=list(np.zeros(18))
df1.loc[len(all_index)]=last_row
df1.loc[len(all_index)+1]=last_row
df1.loc[len(all_index)+2]=last_row
df1['V_X']=np.nan
df1['V_Y']=np.nan
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

[      0       1       2 ... 1048572 1048573 1048574]


In [57]:
v_ego1=df1.loc[df1['Vehicle_ID']==46]
c_index=v_ego1.loc[v_ego1['Lane_ID']==6].index.to_numpy()

In [58]:
df1.iloc[[c_index[0]-1,c_index[0],c_index[0]+1]]

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway
15664,46,619,381,1113433196800,78.881,665.362,6042827.606,2133737.785,16.8,6.4,2,33.88,-4.13,7,0,0,0.00,0.00
15665,46,620,381,1113433196900,78.796,668.767,6042827.083,2133741.253,16.8,6.4,2,33.09,-10.54,6,31,45,42.34,1.28
15666,46,621,381,1113433197000,78.685,671.996,6042826.559,2133744.522,16.8,6.4,2,31.72,-11.20,6,31,45,41.38,1.30


In [71]:
int(df1.loc[(df1['Vehicle_ID']==45) & (df1['Global_Time']==1113433196900)]['Following'])

53

In [54]:
lane=7
filtered_df=df1.loc[df1['Lane_ID']==lane]
list_mergingvehicles=list(filtered_df.Vehicle_ID.unique())
df1_merging_outliers= [5,210,376,388,409,458,465,743,1005,1097,1133,1215,1408,1724,1812,1909,2387,2438,1778,2911]

In [97]:
df1.head()

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0


In [89]:
temp_v_preced_ego=df1.loc[df1['Vehicle_ID']==3364]
#index_v_preced_ego_at_lchange=temp_v_preced_ego.loc[temp_v_preced_ego['Global_Time']==change_Glob_time].index.to_numpy()[0]
temp_v_preced_ego

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway


In [ ]:
dict_interacting_vehicles={}
vehicles_actually_merging=[]
vehicle_row_indices={}
for v_ego in list_mergingvehicles:
    if v_ego not in df1_merging_outliers:
        extra_time_steps=10
        df_ego=df1.loc[df1['Vehicle_ID']==v_ego]
        change_index=df_ego.loc[df_ego['Lane_ID']==6].index.to_numpy()[0] # The index at which lane has changed
        change_Glob_time=int(df1.iloc[change_index]['Global_Time'])
        v_preced_ego=int(df1.iloc[change_index]['Preceding'])
        v_follow_ego=int(df1.iloc[change_index]['Following'])
        #v_follow_ego_follow=int(df1.loc[(df1['Vehicle_ID']==v_follow_ego) & (df1['Global_Time']==change_Glob_time)]['Following'])  
        print(v_preced_ego,v_ego,v_follow_ego)  
        
        ## Indices for ego vehicle
        vehicles_actually_merging.append(v_ego)
        temp=df1.loc[(df1['Vehicle_ID']==v_ego) & (df1['Lane_ID']==7)]
        temp_row_indices_ego=list(temp.loc[temp['Vehicle_ID']==v_ego].index.to_numpy())             

        ############################################################
        ## Indices for vehicle preceding ego - Lets call this VPE ##
        temp_v_preced_ego=df1.loc[df1['Vehicle_ID']==v_preced_ego]
        index_v_preced_ego_at_lchange=temp_v_preced_ego.loc[temp_v_preced_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_preced_ego!=0:            
            if len(index_v_preced_ego_at_lchange)!=0:
                index_VPE_lchange=index_v_preced_ego_at_lchange[0]        
            else:
                # This is when VPE is identified but there is no data for it
                index_VPE_lchange=len(all_index)+1
                print("zero",index_v_preced_ego_at_lchange,index_VPE_lchange)        
        
        else:
            # When ther is no vehicle i.e VPE = 0
            index_VPE_lchange=len(all_index)+1
            print("zero zero")
        
        
        data_index_required_VPE=[]
        for i in range(len(temp_row_indices_ego)):
            if index_VPE_lchange==len(all_index)+1:
                data_index_required_VPE.append(index_VPE_lchange)    
            else:
                data_index_required_VPE.append(index_VPE_lchange-i-1)
        
        #print(data_index_required_VPE)
        
        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VPE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VPE_lchange)    
            else:
                temp_extra_time.append(index_VPE_lchange+i)

        data_index_required_VPE.extend(temp_extra_time)
        data_index_required_VPE.sort()
        
        ############################################################
        
        ############################################################
        ## Indices for vehicle following ego - Lets call this VFE ##
        temp_v_follow_ego=df1.loc[df1['Vehicle_ID']==v_follow_ego]
        index_v_follow_ego_at_lchange=temp_v_follow_ego.loc[temp_v_follow_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_follow_ego!=0:            
            if len(index_v_follow_ego_at_lchange)!=0:
                index_VFE_lchange=index_v_follow_ego_at_lchange[0]        
            else:
                index_VFE_lchange=len(all_index)+1
                print("zero")#,index_VPE_lchange)        
        else:
            index_VFE_lchange=len(all_index)+1
            print("zero zero")

        data_index_required_VFE=[]
        for i in range(len(temp_row_indices_ego)):
            if index_VFE_lchange==len(all_index)+1:
                data_index_required_VFE.append(index_VFE_lchange)    
            else:
                data_index_required_VFE.append(index_VFE_lchange-i-1)

        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VFE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VFE_lchange)    
            else:
                temp_extra_time.append(index_VFE_lchange+i)

        data_index_required_VFE.extend(temp_extra_time)
        data_index_required_VFE.sort()
        ############################################################
        


        
        #print(data_index_required_VFE)
        

In [129]:
####### Filtering data #########
    ####### AND ####### 
#### Calculating Velocities ####
dict_interacting_vehicles={}
vehicles_actually_merging=[]
vehicle_row_indices={}
for v_ego in list_mergingvehicles:
    if v_ego not in df1_merging_outliers:
        
        ## Extra data after merging ##
        extra_time_steps=10
        
        ## Finding interacting cars ## 
        ## For now assume that only cars following and preceeding right after it has merged
        ## the ego vehicle interact with the ego vehicle
        df_ego=df1.loc[df1['Vehicle_ID']==v_ego]
        change_index=df_ego.loc[df_ego['Lane_ID']==6].index.to_numpy()[0] # The index at which lane has changed to 6
        change_Glob_time=int(df1.iloc[change_index]['Global_Time'])
        v_preced_ego=int(df1.iloc[change_index]['Preceding'])
        v_follow_ego=int(df1.iloc[change_index]['Following'])
        dict_interacting_vehicles[str(v_ego)]=[v_preced_ego,v_ego,v_follow_ego]
        #v_follow_ego_follow=int(df1.loc[(df1['Vehicle_ID']==v_follow_ego) & (df1['Global_Time']==change_Glob_time)]['Following'])    
        #v_preced_ego_preced=int(df1.loc[(df1['Vehicle_ID']==v_preced_ego) & (df1['Global_Time']==change_Glob_time)]['Preceding'])
        #dict_interacting_vehicles[str(v_ego)]=[v_preced_ego_preced,v_preced_ego,v_ego,v_follow_ego,v_follow_ego_follow]

        ## Indices for ego vehicle
        vehicles_actually_merging.append(v_ego)
        temp=df1.loc[(df1['Vehicle_ID']==v_ego) & (df1['Lane_ID']==7)]
        data_index_required_E=list(temp.loc[temp['Vehicle_ID']==v_ego].index.to_numpy())             

        ############################################################
        ## Indices for vehicle preceding ego - Lets call this VPE ##
        temp_v_preced_ego=df1.loc[df1['Vehicle_ID']==v_preced_ego]
        index_v_preced_ego_at_lchange=temp_v_preced_ego.loc[temp_v_preced_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_preced_ego!=0:            
            if len(index_v_preced_ego_at_lchange)!=0:
                index_VPE_lchange=index_v_preced_ego_at_lchange[0]        
            else:
                # This is when VPE is identified but there is no data for it
                index_VPE_lchange=len(all_index)+1
                print("zero",index_v_preced_ego_at_lchange,index_VPE_lchange)        
        
        else:
            # When ther is no vehicle i.e VPE = 0
            index_VPE_lchange=len(all_index)+1
            print("zero zero")
        
        
        data_index_required_VPE=[]
        for i in range(len(data_index_required_E)):
            if index_VPE_lchange==len(all_index)+1:
                data_index_required_VPE.append(index_VPE_lchange)    
            else:
                data_index_required_VPE.append(index_VPE_lchange-i-1)
        
        #print(data_index_required_VPE)
        
        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VPE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VPE_lchange)    
            else:
                temp_extra_time.append(index_VPE_lchange+i)

        data_index_required_VPE.extend(temp_extra_time)
        data_index_required_VPE.sort()
        
        ############################################################
        
        ############################################################
        ## Indices for vehicle following ego - Lets call this VFE ##
        temp_v_follow_ego=df1.loc[df1['Vehicle_ID']==v_follow_ego]
        index_v_follow_ego_at_lchange=temp_v_follow_ego.loc[temp_v_follow_ego['Global_Time']==change_Glob_time].index.to_numpy()
        #print(index_v_preced_ego_at_lchange)
        if v_follow_ego!=0:            
            if len(index_v_follow_ego_at_lchange)!=0:
                index_VFE_lchange=index_v_follow_ego_at_lchange[0]        
            else:
                index_VFE_lchange=len(all_index)+1
                print("zero")#,index_VPE_lchange)        
        else:
            index_VFE_lchange=len(all_index)+1
            print("zero zero")

        data_index_required_VFE=[]
        for i in range(len(data_index_required_E)):
            if index_VFE_lchange==len(all_index)+1:
                data_index_required_VFE.append(index_VFE_lchange)    
            else:
                data_index_required_VFE.append(index_VFE_lchange-i-1)

        temp_extra_time=[]
        
        for i in range(extra_time_steps):
            if index_VFE_lchange==len(all_index)+1:
                temp_extra_time.append(index_VFE_lchange)    
            else:
                temp_extra_time.append(index_VFE_lchange+i)

        data_index_required_VFE.extend(temp_extra_time)
        data_index_required_VFE.sort()
        ############################################################
        
        # Adding extra time steps after merging - for EGO vehicle
        temp_extra_time_ego=[]
        for i in range(extra_time_steps):
            temp_extra_time_ego.append(data_index_required_E[len(data_index_required_E)-1]+i)
        data_index_required_E.extend(temp_extra_time_ego)
        
        key=str(v_ego)
        vehicle_row_indices[key]=[data_index_required_VPE,data_index_required_E,data_index_required_VFE]

        # Velocity - v_ego#

        for row_index in data_index_required_E[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
    
        # Velocity - v_preced_ego VPE#

        for row_index in data_index_required_VPE[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
        
        # Velocity - v_follow_ego VFE#

        for row_index in data_index_required_VFE[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)  
    


zero zero
zero zero
zero
zero [] 1048576
zero
zero [] 1048576
zero


In [141]:


# Train test split  
vehicles_for_training=[]
vehicles_for_testing=[]

index=0
for vehicle in vehicles_actually_merging:
    if vehicle not in vehicles_for_training:
        if index%10==0:
            vehicles_for_testing.append(vehicle)
            index+=1
        else:
            vehicles_for_training.append(vehicle)
            index+=1


dict_taining_data={}
for vehicle in vehicles_for_training:
    key=str(vehicle)
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]

    for index_VPE,index_E,index_VFE in zip(data_index[0][1:],data_index[1][1:],data_index[2][1:]):
        temp_traj_point=[df1.iat[index_VPE,4],df1.iat[index_VPE,5],df1.iat[index_VPE,18],df1.iat[index_VPE,19]]
        temp_traj_point.extend([df1.iat[index_E,4],df1.iat[index_E,5],df1.iat[index_E,18],df1.iat[index_E,19]])
        temp_traj_point.extend([df1.iat[index_VFE,4],df1.iat[index_VFE,5],df1.iat[index_VFE,18],df1.iat[index_VFE,19]])
        ref_trajectory.append(temp_traj_point)
    dict_taining_data[key]=ref_trajectory    

dict_test_data={}
for vehicle in vehicles_for_testing:
    key=str(vehicle)
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]

    for index_VPE,index_E,index_VFE in zip(data_index[0][1:],data_index[1][1:],data_index[2][1:]):
        temp_traj_point=[df1.iat[index_VPE,4],df1.iat[index_VPE,5],df1.iat[index_VPE,18],df1.iat[index_VPE,19]]
        temp_traj_point.extend([df1.iat[index_E,4],df1.iat[index_E,5],df1.iat[index_E,18],df1.iat[index_E,19]])
        temp_traj_point.extend([df1.iat[index_VFE,4],df1.iat[index_VFE,5],df1.iat[index_VFE,18],df1.iat[index_VFE,19]])
        ref_trajectory.append(temp_traj_point)
    dict_test_data[key]=ref_trajectory   


In [143]:
dict_test_data

{'46': [[57.815,
   539.671,
   0.91,
   24.36,
   87.052,
   338.33,
   -4.96,
   55.0,
   67.96,
   501.261,
   -0.01,
   22.24],
  [57.983,
   542.131,
   1.68,
   24.6,
   86.566,
   343.687,
   -4.86,
   53.57,
   67.96,
   503.578,
   0.0,
   23.17],
  [58.234,
   544.596,
   2.51,
   24.65,
   86.579,
   349.187,
   0.13,
   55.0,
   67.959,
   505.926,
   -0.01,
   23.48],
  [58.56,
   547.045,
   3.26,
   24.49,
   86.594,
   355.187,
   0.15,
   60.0,
   67.959,
   508.224,
   0.0,
   22.98],
  [58.865,
   549.493,
   3.05,
   24.48,
   86.455,
   359.969,
   -1.39,
   47.82,
   67.958,
   510.424,
   -0.01,
   22.0],
  [59.17,
   551.942,
   3.05,
   24.49,
   86.305,
   365.004,
   -1.5,
   50.35,
   67.959,
   512.577,
   0.01,
   21.53],
  [59.475,
   554.391,
   3.05,
   24.49,
   86.139,
   370.279,
   -1.66,
   52.75,
   67.958,
   514.781,
   -0.01,
   22.04],
  [59.78,
   556.839,
   3.05,
   24.48,
   85.994,
   375.685,
   -1.45,
   54.06,
   67.957,
   517.079,
  